In [1]:
# Main setup
import imagej
ij = imagej.init(['io.github.mianalysis:mia-plugin:2.0.0-SNAPSHOT'])

from jpype import JClass
from scyjava import jimport
from src.utilities.imagerenderer import NotebookImageRenderer
from src.wrappers.coordinatesetwrapper import CoordinateSetFactory
from src.wrappers.imagewrapper import ImageWrapper
from src.wrappers.metadatawrapper import MetadataWrapper
from src.wrappers.objwrapper import ObjFactory
from src.wrappers.objswrapper import ObjsFactory
from src.wrappers.volumewrapper import VolumeFactory
from src.wrappers.workspacewrapper import WorkspaceWrapper

from src.modules.demomodule import ModuleDemo

import time

# Setting renderer
ImageI = jimport('io.github.mianalysis.mia.object.image.ImageI')
ImageI.setGlobalRenderer(NotebookImageRenderer(ij))
ImageI.setUseGlobalImageRenderer(True)

# Setting objects factory
ObjsFactories = jimport('io.github.mianalysis.mia.object.ObjsFactories')
objs_factory = ObjsFactory()
ObjsFactories.addFactory(objs_factory)
ObjsFactories.setDefaultFactory(objs_factory)

# Setting object factory
ObjFactories = jimport('io.github.mianalysis.mia.object.coordinates.ObjFactories')
obj_factory = ObjFactory()
ObjFactories.addFactory(obj_factory)
ObjFactories.setDefaultFactory(obj_factory)

# Setting volume factory
VolumeFactories = jimport('io.github.mianalysis.mia.object.coordinates.volume.VolumeFactories')
volume_factory = VolumeFactory()
VolumeFactories.addFactory(volume_factory)
VolumeFactories.setDefaultFactory(volume_factory)

# Setting coordinateset factory
CoordinateSetFactories = jimport('io.github.mianalysis.mia.object.coordinates.volume.CoordinateSetFactories')
coordinate_set_factory = CoordinateSetFactory()
CoordinateSetFactories.addFactory(coordinate_set_factory)
CoordinateSetFactories.setDefaultFactory(coordinate_set_factory)

# Module imports
FilterImage = jimport('io.github.mianalysis.mia.module.images.process.FilterImage')
IdentifyObjects = jimport('io.github.mianalysis.mia.module.objects.detect.IdentifyObjects')
ImageLoader = jimport('io.github.mianalysis.mia.module.inputoutput.ImageLoader')
ShowImage = jimport('io.github.mianalysis.mia.module.visualise.ShowImage')
ThresholdImage = jimport('io.github.mianalysis.mia.module.images.process.threshold.GlobalAutoThreshold')
AvailableModules = jimport('io.github.mianalysis.mia.module.AvailableModules')

# # Example of adding modules
# print(AvailableModules.getModuleNames(True).size())
# AvailableModules.addModuleName(ModuleDemo(None))
# print(AvailableModules.getModuleNames(False).size())

# # Example of loading a module through reflection
# cl = JClass(str(AvailableModules.getModuleNames(False).get(42)))
# cc = cl(None) # The module constructor
# print(cc.getShortDescription()) # Showing it's worked

OpenJDK 64-Bit Server VM warning: Attempt to protect stack guard pages failed.
OpenJDK 64-Bit Server VM warning: Attempt to deallocate stack guard pages failed.
ScriptEngineManager providers.next(): javax.script.ScriptEngineFactory: Provider org.scijava.plugins.scripting.scala.ScalaScriptLanguage could not be instantiated
ScriptEngineManager providers.next(): javax.script.ScriptEngineFactory: Provider org.scijava.plugins.scripting.scala.ScalaScriptLanguage could not be instantiated
ScriptEngineManager providers.next(): javax.script.ScriptEngineFactory: Provider org.scijava.plugins.scripting.scala.ScalaScriptLanguage could not be instantiated
ScriptEngineManager providers.next(): javax.script.ScriptEngineFactory: Provider org.scijava.plugins.scripting.scala.ScalaScriptLanguage could not be instantiated
[ERROR] Cannot create plugin: org.scijava.plugins.scripting.scala.ScalaScriptLanguage


[INFO] Overriding MIA; identifier: command:io.github.mianalysis.mia.MIA_; jar: file:/Users/sc13967/.jgo/io.github.mianalysis/mia-plugin/2.0.0-SNAPSHOT/d2fe3ba7f0550a523403e0d047b1788b5d374c2a09c72a0b89f635395f421067/mia-plugin-2.0.0-SNAPSHOT.jar
[INFO] Overriding MIA (headless); identifier: command:io.github.mianalysis.mia.MIAHeadless; jar: file:/Users/sc13967/.jgo/io.github.mianalysis/mia-plugin/2.0.0-SNAPSHOT/d2fe3ba7f0550a523403e0d047b1788b5d374c2a09c72a0b89f635395f421067/mia-core-2.0.0-SNAPSHOT.jar


In [2]:
input_path = "/Users/sc13967/Documents/Programming/Python/mia-python/Ex3_Skeletonisation.tif"
series_number = 1
image_name = "demo"
objects_name = "DNA"

# Defining WorkspaceWrapper as WorkspaceI@WorkspaceWrapper seems to allow default method access, 
# although this doesn't appear to support default methods with arguments

# workspace = WorkspaceWrapper(1, input_path, series_number, None)
WorkspaceI = jimport('io.github.mianalysis.mia.object.WorkspaceI')
workspace = WorkspaceI@WorkspaceWrapper(1, input_path, series_number, None)

imageLoader = ImageLoader(None)
imageLoader.updateParameterValue(ImageLoader.OUTPUT_IMAGE, image_name)
imageLoader.setShowOutput(False)
imageLoader.process(workspace)

filterImage = FilterImage(None)
filterImage.updateParameterValue(FilterImage.INPUT_IMAGE, image_name)
filterImage.updateParameterValue(FilterImage.FILTER_MODE, FilterImage.FilterModes.GAUSSIAN2D)
filterImage.updateParameterValue(FilterImage.FILTER_RADIUS, 5)
filterImage.setShowOutput(False)
filterImage.process(workspace)

thresholdImage = ThresholdImage(None)
thresholdImage.updateParameterValue(ThresholdImage.INPUT_IMAGE, image_name)
thresholdImage.setShowOutput(False)
thresholdImage.process(workspace)

<java object 'io.github.mianalysis.mia.object.system.Status'>

In [3]:
identifyObjects = IdentifyObjects(None)
identifyObjects.updateParameterValue(IdentifyObjects.INPUT_IMAGE, image_name)
identifyObjects.updateParameterValue(IdentifyObjects.OUTPUT_OBJECTS, objects_name)
identifyObjects.setShowOutput(True)

import cProfile
import pstats

Exception = jimport('java.lang.Exception')
try:
    t1 = time.time_ns()
    
    with cProfile.Profile() as pr:
        print(identifyObjects.process(workspace))

    stats = pstats.Stats(pr)
    stats.sort_stats(pstats.SortKey.TIME).print_stats(15)

    t2 = time.time_ns()
    
    print(f"Time: {(t2-t1)/1E9}s")
except Exception as ex:
    ex.printStackTrace() 


ObjsFactory: Implement createObjs
ObjsFactory: Implement createObjs
ObjsWrapper: Implement getAndIncrementID


TypeError: Return value is not compatible with required type.